##  Scheduling tester1
Note: uses astropy >= 2.0 and astroplan >= 0.4 and astroplan's FixedTarget.  Numpy is used when possible.

In [2421]:
#!/usr/bin/env python3
"""
Scheduling tester.
"""

'\nScheduling tester.\n'

In [2422]:
import pkg_resources
pkg_resources.require("astropy>=2.0")
pkg_resources.require("astroplan>=0.3")

[astroplan 0.4 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 pytz 2017.2 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 astropy 2.0.2 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 numpy 1.13.3 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 pytest 3.2.1 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 setuptools 36.5.0.post20170921 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages),
 py 1.4.34 (/Users/jdgibson/anaconda3/lib/python3.6/site-packages)]

In [2423]:
# from astroplan import download_IERS_A
# download_IERS_A()

In [2424]:
from astroplan import Observer, FixedTarget
from astropy.time import Time, TimeDelta

In [2425]:
from astroplan import Constraint, AtNightConstraint, AltitudeConstraint
from astroplan import AirmassConstraint, TimeConstraint
from astroplan import MoonSeparationConstraint
from astroplan import ObservingBlock, Transitioner
from astroplan import Slot
from astroplan.constraints import _get_altaz, is_observable
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
import astropy.units as u

import argparse
import numpy as np
import datetime
import requests
import json
import math
from pytz import timezone
import redis
import warnings
import copy
from abc import ABCMeta, abstractmethod

In [2426]:
verbose2 = False
verbose3 = False
verbose4 = True

In [2427]:
# Global constraints
# These parameters enable the constraint
use_AirmassConstraint = True
use_AltitudeConstraint = True
use_AtNightConstraint = True
use_MoonSeparationConstraint = True

# These parameters give detailed outputs for the constraint
test_AirmassConstraint = False
test_AltitudeConstraint = False
test_AtNightConstraint = False
test_MoonSeparationConstraint = True

# Block constraints
# These parameters enable the constraint
use_MaskAngleConstraint = True
use_MaskNumberConstraint = True
use_MeridianConstraint = True
use_PIPriorityConstraint = True
use_RotatorConstraint = True
use_TimeAllocationConstraint = True
use_TimeConstraint = True
# These parameters give detailed outputs for the constraint
test_MaskAngleConstraint = False
test_MaskNumberConstraint = False
test_MeridianConstraint = False
test_PIPriorityConstraint = False
test_RotatorConstraint = False
test_TimeAllocationConstraint = False
test_TimeConstraint = False

test_SequentialScheduler = False

In [2428]:
# Global flag on whether we want to post values to the Redis server.
do_redis = True
# The master Redis server.
redis_host = 'redis.mmto.arizona.edu'

In [2429]:
def get_now():
    return str(datetime.datetime.now())

In [2430]:
get_now()

'2018-01-30 12:26:10.170605'

In [2431]:
def dict_factory(cursor, row):
    d = {}
    for idx,col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [2432]:
import sqlite3
from sqlite3 import Error

"""
CREATE TABLE `scores` (
	`key`	TEXT,
	`value`	REAL,
	PRIMARY KEY(`key`)
)
"""

"""
DELETE from scores;
vacuum scores;
"""

sqlite_file = '/Users/jdgibson/git/QueueScheduler2.0/scores.sqlite' 
table = "scores"

In [2433]:
# create a database connection
conn = sqlite3.connect(sqlite_file)
conn.row_factory = dict_factory

In [2434]:
# This is used for the MaskNumberConstraint
# Always assume that masks 110 and 111 are on Binospec
# masks_used = ['110','111']
masks_used = []

In [2435]:
def get_score(key):
    """

    """
    global conn
    c = conn.cursor()
    c.execute("SELECT * FROM scores WHERE key='{}'".format(key)) 
    row = c.fetchone()
    return row

In [2436]:
def set_score(key, value):
    """

    """
    global conn
    c = conn.cursor()
    
    try:
        sql = "REPLACE INTO scores VALUES ('{}', {})".format(key, value)
        if verbose2:
            print("sql: ", sql)
        
        # Can do this with new versions of sqlite.  It does commits automatically.
        with conn:
            conn.execute(sql)

        # Older version.
        # c.execute(sql)
        # Save (commit) the changes
        # conn.commit()
        # conn.close()
    except sqlite3.IntegrityError:
        print('ERROR: ID already exists in PRIMARY KEY column {}'.format("key"))


In [2437]:
def get_constraint_name(constraint):
    return  type(constraint).__name__

In [2438]:
def is_numeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [2439]:
def get_key(constraint, target, time, time2=None):
    """
    
    """
    constraint_name = get_constraint_name(constraint)
    time.format = 'isot'
    # Typically going with both a start and end time.
    # So, there would be both "time" and "time2" parameters
    if time2 == None:
        key = "{}.{}.{}".format(target.name, 
                        constraint_name,
                        str(time))
    else:
        time2.format = 'isot'
        key = "{}.{}.{}.{}".format(target.name, 
                        constraint_name,
                        str(time),
                        str(time2))
        
    if False:
        print("key:", key)
    return key

In [2440]:
def to_queue(txt):
    global redis_client
    if debug:
        print(txt)
    if mode == 2:
        key = 'DISPATCHER.queue'
    else:
        key = 'SCHEDULER.queue'
    redis_client.set(key, txt)
    redis_client.publish(key, txt)
           
def to_redis(key,value):
    global redis_client
    if debug:
        print(value)
    if do_redis:
        d = {'key':key,
             'value': value,
             'date time': current_datetime(),
             'timestamp': current_milliseconds()}
        d_json = json.dumps(d,sort_keys=True)
        d_json = d_json.replace('\\\"','\"')
        redis_client.set(key, d_json)
        redis_client.publish(key, d_json) 

# This is a helper function to log the outputs from the dispatcher and scheduler
# to a mysql table.  This is done by calling a URL with the correct schedule_id.
# Different CGI parameters are used for the dispatcher and the scheduler.
def to_mysql(sid):
    txt = ""
    try:
        if mode == 2:
            url = 'https://ops.mmto.arizona.edu/ObservatoryManager/QueueSchedules/notify.php?message=complete&type=DISPATCHER&schedule_id=' + str(sid)
        else:
            url = 'https://ops.mmto.arizona.edu/ObservatoryManager/QueueSchedules/notify.php?message=complete&type=SCHEDULER&schedule_id=' + str(sid)
        txt = "url = " + url
        r = requests.get(url)
    except:
        txt = "Error logging to MySQL via URL."
    to_stdout("Logging to mysql via URL:" + txt)


def to_stdout(txt):
    if mode == 2:
        key = 'DISPATCHER.stdout'
    else:
        key = 'SCHEDULER.stdout'
    if True:
        print("In to_stdout: ", key, ": ", txt)
    to_redis(key,txt)

def to_status(txt):
    if mode == 2:
        key = 'DISPATCHER.status'
    else:
        key = 'SCHEDULER.status'
    to_redis(key,txt)

def to_output(txt,schedule_id=None):
    """ Utility function to set/publish to Redis the completed time by program.
    The scheduler details is a large JSON structure that contains all of the
    computational subproducts during scheduling. This includes each constraint score
    for each observing block for each time slot. It is similar to the dispatcher
    output, but for the entire scheduling run.
    """
    if mode == 2:
        if schedule_id is not None:
            key = "DISPATCHER.{}.output".format(schedule_id)
            to_redis(key,txt)
        key = 'DISPATCHER.output'
    else:
        if schedule_id is not None:
            key = "SCHEDULER.{}.output".format(schedule_id)
            to_redis(key,txt)
        key = 'SCHEDULER.output'
    to_redis(key,txt)
        
def to_constraint_details(txt):
    """ Utility function to set/publish to Redis the constraint details for an entire schedule.
    The SCHEDULER.details Redis parameter is a large JSON structure that contains all of the
    computational subproducts during scheduling. This includes each constraint score
    for each observing block for each time slot. It is similar to the dispatcher
    output, but for the entire scheduling run.
    """
    key = 'SCHEDULER.details'
    to_redis(key,txt)

def to_allocated_time(txt):
    """ Utility function to set/publish to Redis the allocated time by program.
    The DISPATCHER.allocated_time and SCHEDULER.allocated_time Redis parameters are
    JSON structures of the allocated time for each program.  The allocated time is 
    used to compute a priority for each observing block, based upon how much time
    each program has already used.
    
    """
    if mode == 2:
        key = 'DISPATCHER.allocated_time'
    else:
        key = 'SCHEDULER.allocated_time'
    to_redis(key,txt)

def to_completed_time(txt):
    """ Utility function to set/publish to Redis the completed time by program.
    The DISPATCHER.allocated_time and SCHEDULER.allocated_time Redis parameters are
    JSON structures of the allocated time for each program.  The allocated time is 
    used to compute a priority for each observing block, based upon how much time
    each program has already used. 
    """
    if mode == 2:
        key = 'DISPATCHER.completed_time'
    else:
        key = 'SCHEDULER.completed_time'
    to_redis(key,txt)
    
def to_exception(txt):
    """ Utility function to set/publish to Redis any run-time exceptions.
    """
    if mode == 2:
        key = 'DISPATCHER.exception'
    else:
        key = 'SCHEDULER.exception'
    to_redis(key,txt)


In [2441]:
def get_config_json(queue_id):
    url = 'https://scheduler.mmto.arizona.edu/QueueSchedules/config_json.php?formatted=0&schedule_id='
    url += str(queue_id)
    r = requests.get(url)
    txt = r.text
    txt = txt.replace("<pre>\n","")
    txt = txt.replace("</pre>\n","")
    txt = txt.replace("\s+","")
    txt = txt.replace("\n","")
    txt = txt.replace("&amp;&amp;","")
    if False:
        print(txt)
    obj = json.loads(txt)
    return obj

In [2442]:
# schedule_id = 804, December binospec run
# queue_id = 804
# schedule_id = 694, Binospec Commissioning Nov Run
# queue_id = 694
queue_id = 812

In [2443]:
def read_setup():
    global schedule_id
    global debug
    global verbose
    global instrument
    global queuerun_id
    global meta
    global allocation
    global stats
    global configuration
    global configs
    global fields
    global start_date
    global end_date
    
    meta = get_config_json(queue_id)
    if False:
        txt = json.dumps(meta, indent=2)
        print(txt)

In [2444]:
    schedule_id = meta["schedule_id"]
    debug = meta["debug"]
    verbose = meta["verbose"]
    instrument = meta['instrument']
    queuerun_id = meta["queuerun_id"]

In [2445]:
    allocation = meta['allocation']
    if False:
        txt = json.dumps(allocation, indent=2)
        print(txt)

In [2446]:
    stats = meta['stats']
    if False:
        txt = json.dumps(stats, indent=2)
        print(txt)

In [2447]:
    configuration = meta['configuration']
    if False:
        txt = json.dumps(configuration, indent=2)
        print(txt)

In [2448]:
    configs = {}
    for conf in configuration:
        if False:
            print(conf, ": " ,  repr(configuration[conf]))
        configs[conf] = configuration[conf]['parametervalue']
        if True:
            print (conf, ": ", configs[conf])

start_date :  2018-02-06 19:00:00
end_date :  2018-02-19 19:00:00
mode :  1
trimester :  2018a
use_airmass_constraint :  true
use_altitude_constraint :  true
use_altitude_boolean :  false
use_at_night_constraint :  true
use_meridian_constraint :  true
use_moon_illumination_constraint :  false
use_moon_separation_constraint :  true
use_programmatic_constraint :  false
use_rotator_constraint :  true
use_time_constraint :  true
use_time_allocation_constraint :  true
use_time_allocation_boolean :  false
use_minimal_constraints :  false
max_solar_altitude :  -12
max_seeing :  5
min_seeing :  0.1
max_good_seeing :  1
min_poor_seeing :  1.4
max_airmass :  2.5
max_alt_degrees :  88
min_alt_degrees :  20
max_rot_degrees :  180
min_rot_degrees :  -164
moon_separation_degrees :  15
slew_duration_seconds :  60
slew_rate :  1
gap_time_hours :  0.5
longslit_overhead_seconds :  1800
mask_overhead_seconds :  1800
imaging_overhead_seconds :  300
time_resolution_seconds :  20
do_mysql :  false
max_field

In [2449]:
    if 'start_date' in configs:
        try:
            start_date = Time(configs['start_date'])
        except:
            start_date = Time('2016-07-06 19:00')
    else:
        start_date = Time('2016-07-06 19:00')

    if 'end_date' in configs:
        try:
            end_date = Time(configs['end_date'])
        except:
            end_date = Time('2016-07-07 19:00')
    else:
        end_date = Time('2016-07-07 19:00')

    if True:
        print("start_date: ", repr(start_date))
        print("end_date: ", repr(end_date))

start_date:  <Time object: scale='utc' format='iso' value=2018-02-06 19:00:00.000>
end_date:  <Time object: scale='utc' format='iso' value=2018-02-19 19:00:00.000>


In [2450]:
    fields = meta['fields']
    if False:
        txt = json.dumps(fields, indent=2)
        print(txt)

In [2451]:
def get_name(f):
    try:
        # This version has the mask_id in the key name.
        if 'mask_id' in f:
            my_name = f['objid'] + '*' + f['mask_id'] + "*" + f['block_id'] + "*" + f['program'] +  "*P" + str(f['pi_priority'])
        # While this version does not have the mask_id (for older queues)
        else:
            my_name = f['objid'] + "*" + f['block_id'] + "*" + f['program'] +  "*P" + str(f['pi_priority'])
    except:
        # Really shouldn't get here...
        my_name = ""

    if False:
        print("my_name: ", my_name)
    return my_name

In [2452]:
# Extracts the mask_id out of the target name
def get_mask_id(name):
    arr = name.split('*')
    # Need to have two versions: one that includes the mask_id 
    # within the name and the other that doesn't (and fails)
    if len(arr) == 5:
        mask_id = arr[1]
    else:
        print("Key name does not include mask_id.  Length should be 5")
        mask_id = 0
        
    if False:
        print("mask_id: ", mask_id)
    return mask_id

In [2453]:
# Extracts the block_id out of the target name
def get_block_id(name):
    arr = name.split('*')
    if len(arr) == 5:
        block_id = arr[2]
    elif len(arr) == 4:
        block_id = arr[1]
    else:
        print("Key name does not include block_id.  Length should be 4 or 5")
        block_id = 0
        
    if False:
        print("block_id: ", block_id)
    return block_id

In [2454]:
# Extracts the program out of the target name
def get_program(name):
    arr = name.split('*')
    program = arr[3] 
    
    if len(arr) == 5:
        program = arr[3]
    elif len(arr) == 4:
        program = arr[2]
    else:
        print("Key name does not include program.  Length should be 4 or 5")
        program = 0
        
    if False:
        print("program: ", program)
    return program

In [2455]:
def add_observer():
    mmto = Observer(longitude=249.11499999999998*u.deg,
                                 latitude=31.688333333333333*u.deg, 
                                 elevation=2608*u.m,
                                 name="mmto",
                                 timezone="America/Phoenix")
    times = Time(["2017-08-01 06:00", "2017-08-01 12:00", "2017-08-01 18:00"])

In [2456]:
    print(mmto)

<Observer: name='mmto',
    location (lon, lat, el)=(-110.88500000000002 deg, 31.688333333333325 deg, 2607.999999999073 m),
    timezone=<DstTzInfo 'America/Phoenix' LMT-1 day, 16:32:00 STD>>


In [2457]:
    print(get_now())

2018-01-30 12:26:10.634796


In [2458]:
# Read in the table of targets
# from astropy.io import ascii
# target_table = ascii.read('targets.txt')
#targets = [FixedTarget(coord=SkyCoord(ra=ra*u.deg, dec=dec*u.deg), name=name)
#           for name, ra, dec in target_table]

In [2459]:
# General function to round a number up to a multiple of "interval".
def roundup(x, interval):
    """ Utility function to round a float number up to the next higher integer.
    """
    return int(math.ceil(float(x) / float(interval)) * interval)

In [2460]:
# Define a set a new Astroplan Constraints.
# These include:
#   1) MaskAngleConstraint
#   2) MaskNumberConstraint
#   3) MeridianConstraint
#   4) PIPriorityConstraint
#   5) RotatorConstraint
#   6) TimeAllocationConstraint
#   7) TimeConstraint (modified to handle multiple time interval)
#
#   The Constraints imported from Astroplan include:
#   1) AirmassConstraint
#   2) AltitudeConstraint
#   3) AtNightConstraint

# A total of ten Constraints.

In [2461]:
class MaskAngleConstraint(Constraint):
    """
       MaskAngleConstraint.

    """
    def __init__(self, observer,
                 design_parang=0.0*u.deg, 
                 max_mask_angle=30.0*u.deg, 
                 grid_times_targets=False, 
                 debug=False):
        if False:
            print("MaskAngleConstraint init")
        try:
            design_parang = Angle(self.design_parang)
        except:
            design_parang = Angle(0 * u.deg)

        try:
            max_mask_angle = Angle(self.max_mask_angle)
        except:
            max_mask_angle = Angle(30 * u.deg)
        self.observer = observer
        self.design_parang = design_parang
        self.max_mask_angle = max_mask_angle
        self.grid_times_targets = grid_times_targets
        self.debug = debug
    
    
    def get_score(self, target, time):
        if False:
            print("target: ", repr(target), ", time: ", repr(time))
                   
        pa = self.observer.parallactic_angle(time, target)
        da = self.design_parang
        ma = self.max_mask_angle
        if False:
            print("pa: ", repr(pa))
        if False:
            print("dp: ", repr(da))
        if False:
            print("ma: ", repr(ma))          
        ang = abs(pa - da) <= abs(ma)
        if ang == True:
            score = 1.0
        else:
            score = 0.1                      
        return score                
    
    def compute_constraint(self, times, observer, targets):
        if False:
            print("In MaskAngleContraint compute_constraint")
        mask = []
        if targets.isscalar:
            target = targets
            for time in times:
                score = self.get_score(target,time)
                if False:
                    print("Scalar MaskAngleConstraint score: ", repr(score))
                mask.append(score)
            if False:
                print("Handling as scalar")
            # Turn the mask into a numpy array
            mask_numpy = np.array(mask)
            # I don't see anyway to implement the grid_times_targets here.
            # If the targets are scalar, the result will just a 1-D array.
            # It will broadcast as need by numpy for any further calculations.
        else:
            for target in targets:
                for time in times:
                    score = self.get_score(target,time)
                    if False:
                        print("Iterater MaskAngleConstraint score: ", repr(score))
                    mask.append(score)
            if False:
                print("Handling as iterable")
            # Turn the mask into a numpy array and reshape.
            mask_numpy = np.reshape(np.array(mask),[len(targets), len(times)]) 
        
        if False:
            print("targets")
            print(repr(targets))
            print("times")
            print(repr(times))
            print("mask")
            print(repr(mask))
            print("mask_numpy")
            print(repr(mask_numpy))
            
        return mask_numpy

In [2462]:
class MaskNumberConstraint(Constraint):
    """
       MaskNumberConstraint.

    """
    def __init__(self, mask_id=None, 
                 masks_used=None, 
                 grid_times_targets=False, 
                 debug=False):
        self.mask_id = mask_id
        self.masks_used = masks_used
        self.grid_times_targets = grid_times_targets
        self.debug = debug
    
    def get_score(self, time, target):
        if len(self.masks_used) < 10 or \
            self.mask in self.masks_used:
            # Just a note that we don't want to include this mask_id in 
            # masks_used here since the block may or may not ultimately
            # get scheduled.  "masks_used" will be updated in the scheduler
            # with updated value passed to this class.
            score = 1.0
        else:   
            score = 0.0
        return score
    
    def compute_constraint(self, times, observer, targets):       
        mask = []
        if targets.isscalar:
            target = targets
            for time in times:
                score = self.get_score(target,time)
                mask.append(score)
            mask_numpy = np.array(mask)
        else:
            for target in targets:
                for time in times:
                    score = self.get_score(target,time)
                    mask.append(score)
            mask_numpy = np.reshape(np.array(mask),[len(targets), len(times)])

        if False:
            print("targets", repr(targets))
            print("times", repr(times))
            print("mask_id", repr(self.mask_id))
            print("mask", repr(mask))
 
        return mask_numpy

In [2463]:
# Define an astroplan constraint for the distance of the target from the meridian.
# The returned value is either a boolean [0,1] if the target is outside of an allowed time
# from meridian transit or a float from [0.0:1.0], where the value is 1.0
# when the target is o/typen the meridian to 0.0 when it is at the anti-meridian (12 hours from the meridian)
class MeridianConstraint(Constraint):
    """Constrains the time for targets from meridian transit.

    Principal investigators (PI's) are required to assigned an integer priority from 1 (highest) to 3 (lowest) to each of their targets.
    The targets should be equally divided into the three priorities (i.e., 1, 2, and 3) so that 1/3 of the requested time correspondes to each
    priority.
    This equal division into the three priorities by even time requested is needed to keep scheduling fair for all projects.
    Every effort will be made to observe all targets, but PI's should anticipate that at least part of their priority 3 targets will not be observed because of poor weather or other causes.


    """
    def __init__(self, 
                 duration,
                 pi_priority,
                 mode="sunset",
                 min_alt_degrees=20 * u.deg,
                 max_solar_altitude=-12 * u.deg,
                 grid_times_targets=False, 
                 debug=False):
        """
        Parameters
        ----------
        max : `~astropy.units.Quantity` or `None` (optional)
            Maximum acceptable separation (in decimal hours) between meridian and target (inclusive).
            `None` indicates no constraint of how far the target can be from the meridian.
        boolean_constraint : bool
            If True, the constraint is treated as a boolean (True for within the
            limits and False for outside).  If False, the constraint returns a
            float on [0, 1], where 0 is when the target is on the anti-meridian and
            1 is when the target is on the meridian.
        """
        self.mode = mode 
        self.duration = duration
        self.pi_priority = pi_priority
        self.min_alt_degrees = min_alt_degrees
        self.max_solar_altitude = max_solar_altitude
        self.grid_times_targets = grid_times_targets
        self.debug = debug
                
        try:
            self.pi_priority = float(self.pi_priority)
        except:
            self.pi_priority = 1.0
        
        # 12 hours: the maximum possible time for a target to be from the meridian 
        self.seconds_in_12hrs = 43200     # 12 hours ==> 12 * 60 * 60 = 43200 seconds
        
        # Set up to TimeDelta constatnt values for future use.
        self.dt_0hrs = TimeDelta(0,format='sec')
        self.dt_1hrs = TimeDelta(3600,format='sec')
        self.dt_1_5hrs = TimeDelta(3600*1.5,format='sec')
        self.dt_2hrs = TimeDelta(3600*2.0,format='sec')
        self.dt_2_5hrs = TimeDelta(3600*2.5,format='sec')
        self.dt_3hrs = TimeDelta(3600*3.0,format='sec')
        self.dt_3_5hrs = TimeDelta(3600*3.5,format='sec')
        self.dt_4hrs = TimeDelta(3600*4.0,format='sec')
        
        if False:
            print("MeridianConstraint initialized")
        
    def get_score(self, target, time, observer):
        # We do a series of tests to see if the observing block is 
        # is setting early in the evening.  We want to give high
        # priority to PIPriority == 1 observing blocks that are
        # setting near sunset and that can still be observed.
        #
        # We take into account the duration of observing blocks
        # when doing this special "sunset" mode.
        #
        # Step 1:  Get the time of the previous sunset.
        #       This will be used to see if the time is close to sunset.
        prev_sun_set_time = observer.sun_set_time(time, 
                            which='previous', 
                            horizon=self.max_solar_altitude)

        # Step 2: Calculate the time from the previous sunset.  
        #        This is an indication of how close we are to sunset.
        #        It will be a small number if we are trying to observe
        #        just after sunset.
        td1 = time - prev_sun_set_time 

        # Step 3: Get the time of the next target rise.
        #       This will be used to see if the target is close to rising
        #       in the east in the morning.
        next_target_set = observer.target_set_time(time,target,
                            which="next", 
                            horizon=self.min_alt_degrees )

        # Step 4: Calculate the time from the next target setting.
        #        This number will be a small positive number when 
        #        the target is above the western horizon.
        td2 = next_target_set - time

        # if False and verbose:
        #    print("tx1: {}, sec: {}, tx2: {}, sec: {}".format(tx1, tx1.sec, tx2, tx2.sec))

        # Note: The next three steps use a "sunset" mode where we want to give
        #       high priority to targets that will be setting within the next 2-4
        #       hours.  We use a graded approach for scoring, based on the duration
        #       of the target/observing block.

        # Step 5: This is the "2-hour-target-duration" sunset special case.  
        #       Evaluate for targets/observing blocks that are more the
        #       _2_ hours in duration, and we are within _4_ hours after sunset,
        #       and the target will be setting within _4_ hours.
        #       Only do this for priority 1 targets.
        #       This is our only chance to observe them.
        #       The score is set to 1.0 to give it the maximum chance of being observed.

        if self.mode == "sunset" and \
                td1 <= self.dt_4hrs and td1 > self.dt_0hrs and \
                td2 <= self.dt_4hrs and td2 > self.dt_0hrs and \
                self.duration >= 2.0 * u.hour and \
                self.pi_priority == 1.0:
            score = 1.0
            if verbose:
                print("Sunset special case (>= 2-hr duration), score:",score)

        # Step 6: This is the "1-hour-target-duration" sunset special case.  
        #       Evaluate for targets/observing blocks that are 1-2 hours
        #       in duration, and we are within _3_ hours after sunset,
        #       and the target will be setting within _3_ hours.
        #       Only do this for priority 1 targets.
        #       This is our only chance to observe them.
        #       The score is set to 1.0 to give it the maximum chance of being observed.
        elif self.mode == "sunset" and \
                td1 <= self.dt_3hrs and td1 > self.dt_0hrs and \
                td2 <= self.dt_3hrs and td2 > self.dt_0hrs and \
                self.duration >= 1.0 * u.hour and \
                self.pi_priority == 1.0:
            score = 1.0
            if verbose:
                print("Sunset special case, (>= 1-hour and < 2-hour duration) score:",score)

        # Step 7: This is the "<1-hour-target-duration" sunset special case.  
        #       Evaluate for targets/observing blocks that are <1 hour
        #       in duration, and we are within _2_ hours after sunset,
        #       and the target will be setting within _2_ hours.
        #       Only do this for priority 1 targets.
        #       This is our only chance to observe them.
        #       The score is set to 1.0 to give it the maximum chance of being observed.
        elif self.mode == "sunset" and \
                td1 <= self.dt_2hrs and td1 > self.dt_0hrs and \
                td2 <= self.dt_2hrs and td2 > self.dt_0hrs and \
                self.pi_priority == 1.0:
            score = 1.0
            if verbose:
                print("Sunset special case, (any duration) score:",score)

        # Step 8:  If all of the other conditions have not been true,
        #       Determine how far the target is from the meridian in seconds
        #       and divide by 12 hours (== 43200 seconds) 
        #       The target can be in either rising towards the meridian or
        #       setting away from the meridian.
        else:

            meridian_time = observer.target_meridian_transit_time(time,target,which='nearest')
            diff = abs(time.unix - meridian_time.unix)

            # There are times when the meridian time is 24 hours off.
            # So, the math here accounts for that.
            # If the time difference is more than 24 hours (43200 seconds), 
            # subtract 24 hours.
            if diff > self.seconds_in_12hrs:
                diff -= self.seconds_in_12hrs * 2 # 24 hours in seconds.
                # Recheck that we are using an absolute value.
                diff = abs(diff)

            # Here is the meridian scoring algorithm.  
            # The closer to the meridian, the closer the score is to 1.0.  
            # The range of scores is 1.0 (on the meridian) to 
            # 0.0 (on the anti-meridian).
            score = 1.0 - (diff / self.seconds_in_12hrs)               

            # The score should already range from 0.0 to 1.0.
            if score < 0.0:
                score = 0.0
            if score > 1.0:
                score = 1.0

        if False:
            print("score: ", repr(score))
        return score          
                    
    def compute_constraint(self, times, observer, targets):
        """
        The MeridianConstraint is calculated by: 1) determining the number of hours the target is from the meridian, and 2) calculating a constraint using Math.abs((12. - hours_from_meridan)/12.0) for the target's position at the beginning, middle, and end of the observing block.  
        The calculated scores for these three times will be different.  
        This causes the constraint to equal 1.0 on the meridian and 0.0 on the anti-meridian (12 hours away).  
        Since the absolute value is used, it doesn't matter which direction the target is from the meridan, i.e., positive hours or negative hours.  Values will always vary from 1.0 to 0.0
        It is possible that the target passes through the meridian during the observing block, i.e., it "transits". 
        Caution should be used in cases where the target transits in that azimuth velocities can be very large if the target is close to zenith.
        The maximum AltitudeConstraint should help prevent extremely large azimuth velocities.  

        It should be remembered that constraint scores are calculated at the beginning, middle, and end of each observing block as part of score for the block.
        This causes the constraint to be multiplied by itself three times and the constraint to vary as 1/X^^3 rather than 1/X.

        """
        if False:
            print("MeridianConstraint computing")

        mask = []
        # Testing if targets is scalar
        if targets.isscalar:
            target = targets
            for time in times:
                score = self.get_score(target,time,observer)
                mask.append(score)
            mask_numpy = np.array(mask)
        else:
            for target in targets:
                for time in times:
                    score = self.get_score(target,time,observer)
                    mask.append(score)
            mask_numpy = np.reshape(np.array(mask),[len(targets), len(times)])
    
        return mask_numpy

In [2464]:
class PIPriorityConstraint(Constraint):
    """
       PIPriorityConstraint.

    """
    def __init__(self, pi_priority=1.0, 
                 grid_times_targets=False, 
                 debug=False):
        try:
            self.pi_priority = float(pi_priority)
        except:
            self.pi_priority = 1.0
        self.grid_times_targets = grid_times_targets
        self.debug = debug
       
    def compute_constraint(self, times, observer, targets):             
        # Testing if targets is scalar
        if targets.isscalar:
            mask = [1.0/float(self.pi_priority)
                for time in times]
        else:
            mask = [([1.0/float(self.pi_priority)
                for time in times])
                    for target in targets]

        mask_numpy = np.array(mask)
            
        if False:
            print("targets")
            print(repr(targets))
            print("times")
            print(repr(times))
            print("mask_numpy")
            print(repr(mask_numpy))
 
        return mask_numpy

In [2465]:
class RotatorConstraint(Constraint):
    def __init__(self, max=None, min=None, 
                 posang=None,
                 grid_times_targets=False, 
                 debug=True):
        """
        Parameters
        ----------
        min : `~astropy.units.Quantity` or `None` (optional)
            Minimum acceptable rotator angle (inclusive).
            `None` indicates no limit.
        max : `~astropy.units.Quantity` or `None` (optional)
            Maximum acceptable rotator angle (inclusive).
            `None` indicates no limit.
        """
        if False:
            print("max", repr(max))
            print("min", repr(min))
            print("posang", repr(posang))
        
        if max == None:
            self.max = 180.0 * u.deg
        else:
            self.max = max * u.deg
 
        if min == None:
            self.max = -180.0 * u.deg
        else:
            self.min = min * u.deg

        if posang == None:
            self.posang = 0.0 * u.deg
        # It can't seem to handle 0.0 degrees???
        elif abs(float(posang)) < 0.1:
            self.posang = 0.1 * u.deg
        else:
            self.posang = float(posang) * u.deg

        self.target_posang = Angle(self.posang)    
                
        self.upper_limit = Angle(self.max)
        self.lower_limit =  Angle(self.min)
        
        self.grid_times_targets = grid_times_targets
        self.debug = debug

    def get_score(self,target,time,observer):
        parang = observer.parallactic_angle(time, target)
        # rotator angle = parallactic_angle - position_angle
        # "sky offset" == "position angle"
        # rotator position == rototor angle + rotator offset
        #         
        if False:
            print("parang", repr(parang))
            print("target_posang", repr(self.target_posang))
            
        rot_ang = parang - self.target_posang
        if rot_ang < -180.0 * u.deg:
            rot_ang += 360.0 * u.deg
            
        upper_test =  rot_ang <= self.upper_limit
        # New code to test limits again at 360 degrees rotation.  JDG 2016-12-05
        if not upper_test:
            upper_test = rot_ang - 360.0 * u.deg <= self.upper_limit
            
        lower_test =  self.lower_limit <= rot_ang
        # New code to test limits again at 360 degrees rotation.  JDG 2016-12-05
        if not lower_test:
            lower_test = self.lower_limit <= rot_ang + 360.0 * u.deg

        score = lower_test & upper_test
        return score        
                
    def compute_constraint(self, times, observer, targets):       
        mask = []
        # Testing if targets is scalar
        if targets.isscalar:
            target = targets
            for time in times:
                score = self.get_score(target,time,observer)
                mask.append(score)
            mask_numpy = np.array(mask)
        else:
            for target in targets:
                for time in times:
                    score = self.get_score(target,time,observer)
                    mask.append(score)
            mask_numpy = np.reshape(np.array(mask),[len(targets), len(times)])
            
        if False:
            print("targets")
            print(repr(targets))
            print("times")
            print(repr(times))
            print("mask_numpy")
            print(repr(mask_numpy))
 
        return mask_numpy

In [2466]:
class TimeAllocationConstraint(Constraint):
    """
       TimeAllocationConstraint.

    """
    def __init__(self, program, 
                 stats, 
                 grid_times_targets=False, 
                 debug=False):
        self.program = program
        self.stats = stats
        # "program_hours_allocated" (in hours) from "stats" structure
        self.program_hours_allocated = self.stats[self.program]['program_hours_allocated']
        # "total_hours_used" (in hours) from "stats" structure
        # This will be mutated by this call.
        self.total_hours_used = self.stats[self.program]['total_hours_used']
        self.grid_times_targets = grid_times_targets
        self.debug = debug
    
    def get_score(self, target, time):
        score = 1.0 - (float(self.total_hours_used)/float(self.program_hours_allocated))
        almost_zero = 0.1
        if score < almost_zero:
            score = almost_zero
        elif score > 1.0:
            score = 1.0
        return score
    
    def compute_constraint(self, times, observer, targets):       
        mask = [] 

        if targets.isscalar:
            target = targets
            for time in times:
                score = self.get_score(target,time)
                if False:
                    print("Scalar TimeAllocationConstraint score: ", repr(score))
                mask.append(score)
            if False:
                print("Handling as scalar")
            numpy_mask = np.array(mask)
        else:
            for target in targets:
                for time in times:
                    score = self.get_score(target,time)
                    if False:
                        print("Iterater TimeAllocationConstraint score: ", repr(score))
                    mask.append(score)
            if False:
                print("Handling as iterable")
            # Turn the mask into a numpy array and reshape.
            numpy_mask = np.reshape(np.array(mask),[len(targets), len(times)]) 
        
        if False:
            print("targets")
            print(repr(targets))
            print("times")
            print(repr(times))
            print("mask")
            print(repr(mask))
            print("numpy_mask")
            print(repr(numpy_mask))
            
        return numpy_mask

In [2467]:
class TimeConstraint(Constraint):
    """Constrain the observing time to be within certain time limits.
    An example use case for this class would be to associate an acceptable
    time range with a specific observing block. This can be useful if not
    all observing blocks are valid over the time limits used in calls
    to `is_observable` or `is_always_observable`.
    """

    def __init__(self, min=None, max=None, constraint_times=None):
        """
        Parameters
        ----------
        min : `~astropy.time.Time`
            Earliest time (inclusive). `None` indicates no limit.
        max : `~astropy.time.Time`
            Latest time (inclusive). `None` indicates no limit.
        Examples
        --------
        Constrain the observations to targets that are observable between
        2016-03-28 and 2016-03-30:
        >>> from astroplan import Observer
        >>> from astropy.time import Time
        >>> subaru = Observer.at_site("Subaru")
        >>> t1 = Time("2016-03-28T12:00:00")
        >>> t2 = Time("2016-03-30T12:00:00")
        >>> constraint = TimeConstraint(t1,t2)
        """
        self.min = min
        self.max = max
        self.constraint_times = constraint_times

        if self.constraint_times is None:
            # Original case where there is only a single min and max time
            if self.min is None and self.max is None:
                raise ValueError("You must at least supply either a minimum or a "
                                 "maximum time.")

            if self.min is not None:
                if not isinstance(self.min, Time):
                    raise TypeError("Time limits must be specified as "
                                    "astropy.time.Time objects.")

            if self.max is not None:
                if not isinstance(self.max, Time):
                    raise TypeError("Time limits must be specified as "
                                    "astropy.time.Time objects.")

        else:
            # New case where constraint_times is an array of Time duples,
            # e.g., [[TimeStart1, TimeEnd1],[TimeStart2, TimeEnd2],[ TimeStart3, TimeEnd3]]
            # 
            # Check each start Time and end Time object.
            for [min_t,max_t] in self.constraint_times:
                if min_t is None and max_t is None:
                    raise ValueError("You must at least supply either a minimum or a "
                                     "maximum time.")

                if min_t is not None:
                    if not isinstance(min_t, Time):
                        raise TypeError("Time limits must be specified as "
                                        "astropy.time.Time objects.")

                if max_t is not None:
                    if not isinstance(max_t, Time):
                        raise TypeError("Time limits must be specified as "
                                        "astropy.time.Time objects.")

    def get_score(self,times,min_time,max_time):
        return np.logical_and(times > min_time, times < max_time)
                        
    def compute_constraint(self, times, observer, targets):
        if self.constraint_times is None:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                min_time = Time("1950-01-01T00:00:00") if self.min is None else self.min
                max_time = Time("2120-01-01T00:00:00") if self.max is None else self.max
                # mask = np.logical_and(times > min_time, times < max_time)
                mask = self.get_score(times,min_time,max_time)
            return mask
        else:
            first = True
            for [min_t,max_t] in self.constraint_times:
                with warnings.catch_warnings():
                    warnings.simplefilter('ignore')
                    min_time = Time("1950-01-01T00:00:00") if min_t is None else min_t
                    max_time = Time("2120-01-01T00:00:00") if max_t is None else max_t
                    m2 = self.get_score(times,min_time,max_time)
                    # The first time mask will be None.
                    if first:
                        first = False
                        mask = m2
                    else:
                        # If it's True for any of the possible time periods, it's True 
                        # So use "logical_or".
                        mask = np.logical_or(m2, mask)
            return mask
            

In [2468]:
def add_global_constraints():
    global global_constraints
    #
    # Set up the three global Constraints:
    # 1) AirmassConstraint 
    # 2) AltitudeConstraint
    # 3) AtNightConstraint
    # 4) MoonSeparationConstraint
    #
    # All are used as booleans (0 or 1) in that
    # they either completely pass or fail the constraint.
    # Note that constraints are evaluated at the beginning,
    # middle, and end of each time block.  If a boolean
    # constraint fails any one of these time periods, 
    # it fails overall.
    #

In [2469]:
    # Create the list of constraints that all targets must satisfy
    global_constraints = []

In [2470]:
    #
    # Evaluating AirmassConstraint
    if use_AirmassConstraint and "use_airmass_constraint" in configs and \
        configs['use_airmass_constraint'].lower() == "true" and \
        is_numeric(configs['max_airmass']):
            # If True, return the score as either 1.0 or 0.0, 
            # not as as float.
            #
            # Just assume this is True for now.  We could add this
            # as a configs parameter in the future if we want.
            boolean_constraint = True
            c = AirmassConstraint(max = float(configs['max_airmass']), 
                                    boolean_constraint = boolean_constraint)
            if test_AirmassConstraint:
                print("Adding global constraint: ", repr(c))
            global_constraints.append(c)

In [2471]:
    # 
    # Evaluating AltitudeConstraint
    if use_AltitudeConstraint and "use_altitude_constraint" in configs and \
        configs['use_altitude_constraint'].lower() == "true" and \
        is_numeric(configs['max_alt_degrees']) and \
        is_numeric(configs['min_alt_degrees']) and \
        'use_altitude_boolean' in configs:
            if configs['use_altitude_boolean'].lower() == 'true':
                boolean_constraint = True
            else:
                boolean_constraint = False
            c = AltitudeConstraint(min = configs['min_alt_degrees']*u.deg,
                                    max = configs['max_alt_degrees']*u.deg,
                                    boolean_constraint = boolean_constraint)
            if test_AltitudeConstraint:
                print("Adding global constraint: ", repr(c))
            global_constraints.append(c)

In [2472]:
    #
    # Evaluating AtNightConstraint
    if use_AtNightConstraint and "use_at_night_constraint" in configs and \
        configs['use_at_night_constraint'].lower() == "true" and \
        'max_solar_altitude' in configs and \
        is_numeric(configs['max_solar_altitude']):
            if float(configs['max_solar_altitude']) == -6:
                c = AtNightConstraint.twilight_civil()
            elif float(configs['max_solar_altitude']) == -12:
                c = AtNightConstraint.twilight_civil()
            else:
                c = AtNightConstraint.twilight_astronomical()
            """
            'max_solar_altitude' in configs and \
            is_numeric(configs['max_solar_altitude']):
                pass

                # Only allowing -6 and -12, otherwise use astronomical twilight.
                # We could use max_solar_altitude directly
                if 'max_solar_altitude' in configs and \
                is_numeric(configs['max_solar_altitude']) and \
                float(configs['max_solar_altitude']) == -6:
                    c = AtNightConstraint.twilight_civil()
                elif 'max_solar_altitude' in configs and \ 
                    is_numeric(configs['max_solar_altitude']) and \
                    float(configs['max_solar_altitude']) == -12:
                    c = AtNightConstraint.twilight_nautical()
                else:
                    c = AtNightConstraint.twilight_astronomical()
            """
            if test_AtNightConstraint:
                print("Adding global constraint: ", repr(c))
            global_constraints.append(c)

In [2473]:
    # Evaluating MoonSeparationConstraint
    if use_MoonSeparationConstraint and "use_moon_separation_constraint" in configs and \
        configs['use_moon_separation_constraint'].lower() == "true" and \
        is_numeric(configs['moon_separation_degrees']):
            # If True, return the score as either 1.0 or 0.0, 
            # not as as float.
            #
            # Just assume this is True for now.  We could add this
            # as a configs parameter in the future if we want.
            boolean_constraint = True
            
            c = MoonSeparationConstraint(min=float(configs['moon_separation_degrees'])*u.deg)
            if test_MoonSeparationConstraint:
                print("Adding global constraint: ", repr(c))
            global_constraints.append(c)

Adding global constraint:  <astroplan.constraints.MoonSeparationConstraint object at 0x10ea7b4a8>


In [2474]:
def add_block_constraints():
    #
    # Now we want to process each of the "fields" (an
    # alias for observing block).  The rest of the 
    # constraints will be block-specific.  We use this
    # approach to pass in block-specific parameters
    # to the constraint that are used when the score for
    # the constraint is calculated.
    global blocks

    targets = []
    blocks = []
    for field in fields:
        # Create a name for this observing block.  
        # The name will contain information about the 
        # mask_id, the block_id, program, etc.
        # Encapsulating this information in the name
        # makes the name unique and also allows the
        # information to be extracted later.
        # Astroplan FixedTargets are commonly cast as
        # Astropy SkyCoord.  When this happens, we lose
        # any target-specific information, such as a 
        # mask_id.  This target-specific information
        # can be extracted from the field/observing
        # block name as needed.
        name = get_name(field)
        if False:
            print("name: ", name, "field: ", repr(field))
        r = field['ra_hms']
        d = field['dec_dms']
        if False:
            print("ra: ", repr(r))
            print("dec: ", repr(d))
        #
        # Create a standard Astroplan FixedTarget instance.
        t = FixedTarget(coord=SkyCoord(ra=r, dec=d), name=name)
        targets.append(t)
        if 'time_resolution_seconds' in configs:
            time_resolution_seconds = int(configs['time_resolution_seconds'])
        else:
            time_resolution_seconds = '20'
        # duration

        # The total duration of a field equals the "duration"
        # plus the "overhead".  We may use Astroplan's 
        # TransitionBlocks at a future time.  At the moment,
        # we are using these fixed overheads.
        d = float(field['duration'])
        o = float(field['overhead'])
        # Total duration, "df":  duration plus overhead
        df = roundup(d + o, time_resolution_seconds)*u.second
        if False:
            print("Duration+overhead: ", repr(df))

        # A default priority for this block We're not 
        # using this for PriorityScheduling so it doesn't really
        # matter what the value is.
        p = 1.0

        # Now, add all of the block constraints as needed for the 
        # observing block/field.
        block_constraints = []

        # Evaluating MaskAngleConstraint
        if use_MaskAngleConstraint and \
            'design_parang' in field and \
            'max_mask_angle' in configs:

            if test_MaskAngleConstraint:
                print("Doing MaskAngleConstraint")

            try:
                design_parang = field['design_parang']*u.deg
            except:
                print("design_parang invalid: ", repr(field['design_parang']))
                # Defaults to 0.0 degrees.
                design_parang = 0.0*u.deg

            try:
                max_mask_angle = configs['max_mask_angle']*u.deg
            except:
                print("max_mask_angle invalid: ", repr(configs['max_mask_angle']))
                # Defaults to 30.0 degrees
                max_mask_angle = 30.0*u.deg

            if test_MaskAngleConstraint:
                print("Defining MaskAngleConstraint, p: ", design_parang, ", m: ", design_parang)
            c = MaskAngleConstraint(mmto,
                     design_parang=design_parang, 
                     max_mask_angle=max_mask_angle)

            if test_MaskAngleConstraint:
                print("Adding constraint: ", repr(c))

            block_constraints.append(c) 

        # Evaluating MaskNumberConstraint
        if use_MaskNumberConstraint and "mask_id" in field:
            c = MaskNumberConstraint(field['mask_id'], 
                    masks_used)
            if test_MaskNumberConstraint:
                print("Adding constraint: ", repr(c))
            block_constraints.append(c) 

        #  Evaluating MeridianConstraint    
        if use_MeridianConstraint and 'pi_priority' in field:

            try:
                pp = float(field['pi_priority'])
            except:
                print("pi_priority invalid: ", repr(field['pi_priority']))
                # Defaults to a PIPriority of 1.0, the highest value.
                # We're currently using priorities 1 (highest), 2 and 3 (lowest).
                pp = 1.0

            c = MeridianConstraint(df,
                        pp)
            if test_MeridianConstraint:
                print("Adding constraint: ", repr(c))
            block_constraints.append(c) 

        #  Evaluating PIPriorityConstraint
        if use_PIPriorityConstraint and \
                "use_pi_priority_constraint" in configs and \
                configs['use_pi_priority_constraint'].lower() == "true" and \
                'pi_priority' in field:

            try:
                pp = float(field['pi_priority'])
            except:
                print("pi_priority invalid: ", repr(field['pi_priority']))
                pp = 1.0

            c = PIPriorityConstraint(pi_priority=pp)
            if test_PIPriorityConstraint:
                print("Adding constraint: ", repr(c))
            block_constraints.append(c)

        #  Evaluating RotatorConstraint
        if use_RotatorConstraint and \
                "use_rotator_constraint" in configs and \
                configs['use_rotator_constraint'].lower() == "true" and \
                'max_rot_degrees' in configs and \
                'min_rot_degress' in configs and \
                'posang' in field:

            try:
                max_rot_degrees = float(configs['max_rot_degrees'])
            except:
                print("max_rot_degrees invalid: ", repr(configs['max_rot_degrees']))
                # Defaults to 160 degrees. This should be safe for all
                # instruments.
                max_rot_degrees = 160.0

            try:
                min_rot_degrees = float(configs['min_rot_degrees'])
            except:
                print("min_rot_degrees invalid: ", repr(configs['min_rot_degrees']))
                # Defaults to -160 degrees. This should be safe for all
                # instruments.
                min_rot_degrees = -160.0

            try:
                posang = float(field['posang'])
            except:
                print("posang invalid: ", repr(field['posang']))
                # Position angle defaults to 0.0(degrees)
                posang = 0.0

            c = RotatorConstraint( max=max_rot_degrees,
                     min=min_rot_degrees,
                     posang=posang)

            if test_RotatorConstraint:
                print("Adding constraint: ", repr(c))
            block_constraints.append(c)

        #  Evaluating TimeAllocationConstraint
        if use_TimeAllocationConstraint and \
                "use_time_allocation_constraint" in configs and \
                configs['use_time_allocation_constraint'].lower() == "true" and \
                'program' in field: 

            p = field['program']
            c = TimeAllocationConstraint(p,
                 stats)

            if test_TimeAllocationConstraint:
                print("constraint:", repr(c))
            block_constraints.append(c)

        # Evaluating TimeConstraint
        if use_TimeConstraint and \
                "use_time_constraint" in configs and \
                configs['use_time_constraint'].lower() == "true":

            # Handle case where a list of time_constraints are supplied.
            if 'time_constraints' in field:
                block_times = []
                for [t1,t2] in field['time_constraints']:
                    arr = [Time(t1),Time(t2)]
                    block_times.append(arr)
                    c = (TimeConstraint(None,None,block_times))
                    block_constraints.append(c)
            # Else, case where we have just one time constraint with
            # a single start and end.
            elif 'time_constraint_start' in field and \
                'time_constraint_end' in field:
                c = TimeConstraint(Time(field['time_constraint_start']),
                                        Time(field['time_constraint_end']),
                                        None)
                block_constraints.append(c)

        b = ObservingBlock(t,df,p,constraints=block_constraints)
        if False:
            print("Appending block: ", repr(b))
            print("block_constraints: ", repr(block_constraints))
        blocks.append(b)

In [2475]:
# End of Constraint classes

In [2476]:
def create_transitioner():
    global transitioner
    if 'slew_rate' in configs:
        try:
            slew_rate = float(configs['slew_rate'])* u.deg / u.second
        except:
            print("slew_rate invalid: ", repr(configs['slew_rate']))
            slew_rate = 1.0* u.deg / u.second
    else:
        slew_rate = 1.0* u.deg / u.second

    transitioner = Transitioner(slew_rate,
                                 {'filter':{('B','G'): 10*u.second,
                                            ('G','R'): 10*u.second,
                                            'default': 30*u.second}})

In [2477]:
#
# The Schedule, Scheduler, and SequentialScheduler class are 
# the same as for Astroplan 0.4.  They are slightly modified 
# here so that we can log detailed information about the schedules.

In [2478]:
class Schedule(object):
    """
    An object that represents a schedule, consisting of a list of
    `~astroplan.scheduling.Slot` objects.
    """
    # as currently written, there should be no consecutive unoccupied slots
    # this should change to allow for more flexibility (e.g. dark slots, grey slots)

    def __init__(self, start_time, end_time, constraints=None):
        """
        Parameters
        -----------
        start_time : `~astropy.time.Time`
            The starting time of the schedule; the start of your
            observing window.
        end_time : `~astropy.time.Time`
           The ending time of the schedule; the end of your
           observing window
        constraints : sequence of `~astroplan.constraints.Constraint` s
           these are constraints that apply to the entire schedule
        """
        self.start_time = start_time
        self.end_time = end_time
        self.slots = [Slot(start_time, end_time)]
        self.observer = None

    def __repr__(self):
        return ('Schedule containing ' + str(len(self.observing_blocks)) +
                ' observing blocks between ' + str(self.slots[0].start.iso) +
                ' and ' + str(self.slots[-1].end.iso))

    @property
    def observing_blocks(self):
        return [slot.block for slot in self.slots if isinstance(slot.block, ObservingBlock)]

    @property
    def scheduled_blocks(self):
        return [slot.block for slot in self.slots if slot.block]

    @property
    def open_slots(self):
        return [slot for slot in self.slots if not slot.occupied]

    def to_table(self, show_transitions=True, show_unused=False):
        # TODO: allow different coordinate types
        target_names = []
        start_times = []
        end_times = []
        durations = []
        ra = []
        dec = []
        config = []
        for slot in self.slots:
            if hasattr(slot.block, 'target'):
                start_times.append(slot.start.iso)
                end_times.append(slot.end.iso)
                durations.append(slot.duration.to(u.minute).value)
                target_names.append(slot.block.target.name)
                ra.append(slot.block.target.ra)
                dec.append(slot.block.target.dec)
                config.append(slot.block.configuration)
            elif show_transitions and slot.block:
                start_times.append(slot.start.iso)
                end_times.append(slot.end.iso)
                durations.append(slot.duration.to(u.minute).value)
                target_names.append('TransitionBlock')
                ra.append('')
                dec.append('')
                changes = list(slot.block.components.keys())
                if 'slew_time' in changes:
                    changes.remove('slew_time')
                config.append(changes)
            elif slot.block is None and show_unused:
                start_times.append(slot.start.iso)
                end_times.append(slot.end.iso)
                durations.append(slot.duration.to(u.minute).value)
                target_names.append('Unused Time')
                ra.append('')
                dec.append('')
                config.append('')
        return Table([target_names, start_times, end_times, durations, ra, dec, config],
                     names=('target', 'start time (UTC)', 'end time (UTC)',
                            'duration (minutes)', 'ra', 'dec', 'configuration'))

    def new_slots(self, slot_index, start_time, end_time):
        """
        Create new slots by splitting a current slot.
        Parameters
        ----------
        slot_index : int
            The index of the slot to split
        start_time : `~astropy.time.Time`
            The start time for the slot to create
        end_time : `~astropy.time.Time`
            The end time for the slot to create
        Returns
        -------
        new_slots : list of `~astroplan.scheduling.Slot` s
            The new slots created
        """
        # this is intended to be used such that there aren't consecutive unoccupied slots
        new_slots = self.slots[slot_index].split_slot(start_time, end_time)
        return new_slots

    def insert_slot(self, start_time, block):
        """
        Insert a slot into schedule and associate a block to the new slot.
        Parameters
        ----------
        start_time : `~astropy.time.Time`
            The start time for the new slot.
        block : `~astroplan.scheduling.ObservingBlock`
            The observing block to insert into new slot.
        Returns
        -------
        slots : list of `~astroplan.scheduling.Slot` objects
            The new slots in the schedule.
        """
        # due to float representation, this will change block start time
        # and duration by up to 1 second in order to fit in a slot
        for j, slot in enumerate(self.slots):
            if ((slot.start < start_time or abs(slot.start-start_time) < 1*u.second)
                    and (slot.end > start_time + 1*u.second)):
                slot_index = j
        if (block.duration - self.slots[slot_index].duration) > 1*u.second:
            # raise is causing errors.  Try to ignore it...  JDG  2018-01-26
            if False:
                raise ValueError('longer block than slot')
        elif self.slots[slot_index].end - block.duration < start_time:
            start_time = self.slots[slot_index].end - block.duration

        if abs((self.slots[slot_index].duration - block.duration)) < 1 * u.second:
            # slot duration is very similar to block duration.
            # force equality so block fits
            block.duration = self.slots[slot_index].duration
            start_time = self.slots[slot_index].start
            end_time = self.slots[slot_index].end
        elif abs(self.slots[slot_index].start - start_time) < 1*u.second:
            # start time of block is very close to slot start time
            # force equality to avoid tiny gaps
            start_time = self.slots[slot_index].start
            end_time = start_time + block.duration
        elif abs(self.slots[slot_index].end - start_time - block.duration) < 1*u.second:
            # end time is very close to slot end time
            # force equality to avoid tiny gaps
            end_time = self.slots[slot_index].end
        else:
            end_time = start_time + block.duration

        if isinstance(block, ObservingBlock):
            # TODO: make it shift observing/transition blocks to fill small amounts of open space
            block.end_time = start_time+block.duration
        earlier_slots = self.slots[:slot_index]
        later_slots = self.slots[slot_index+1:]
        block.start_time = start_time
        new_slots = self.new_slots(slot_index, start_time, end_time)
        for new_slot in new_slots:
            if new_slot.middle:
                new_slot.occupied = True
                new_slot.block = block
        self.slots = earlier_slots + new_slots + later_slots
        return earlier_slots + new_slots + later_slots

    def change_slot_block(self, slot_index, new_block=None):
        """
        Change the block associated with a slot.
        This is currently designed to work for TransitionBlocks in PriorityScheduler
        The assumption is that the slot afterwards is open and that the start time
        will remain the same.
        If the block is changed to None, the slot is merged with the slot
        afterwards to make a longer slot.
        Parameters
        ----------
        slot_index : int
            The slot to edit
        new_block : `~astroplan.scheduling.TransitionBlock`, default None
            The new transition block to insert in this slot
        """
        if self.slots[slot_index + 1].block:
            raise IndexError('slot afterwards is full')
        if new_block is not None:
            new_end = self.slots[slot_index].start + new_block.duration
            self.slots[slot_index].end = new_end
            self.slots[slot_index].block = new_block
            self.slots[slot_index + 1].start = new_end
            return slot_index
        else:
            self.slots[slot_index + 1].start = self.slots[slot_index].start
            del self.slots[slot_index]
            return slot_index - 1

In [2479]:
# This is the Scheduler class from astroplan 0.4
class Scheduler(object):
    """
    Schedule a set of `~astroplan.scheduling.ObservingBlock` objects
    """

    __metaclass__ = ABCMeta

    @u.quantity_input(gap_time=u.second, time_resolution=u.second)
    def __init__(self, constraints, observer, transitioner=None,
                 gap_time=5*u.min, time_resolution=20*u.second):
        """
        Parameters
        ----------
        constraints : sequence of `~astroplan.constraints.Constraint`
            The constraints to apply to *every* observing block.  Note that
            constraints for specific blocks can go on each block individually.
        observer : `~astroplan.Observer`
            The observer/site to do the scheduling for.
        transitioner : `~astroplan.scheduling.Transitioner` (required)
            The object to use for computing transition times between blocks.
            Leaving it as ``None`` will cause an error.
        gap_time : `~astropy.units.Quantity` with time units
            The maximum length of time a transition between ObservingBlocks
            could take.
        time_resolution : `~astropy.units.Quantity` with time units
            The smallest factor of time used in scheduling, all Blocks scheduled
            will have a duration that is a multiple of it.
        """
        self.constraints = constraints
        self.observer = observer
        self.transitioner = transitioner
        if not isinstance(self.transitioner, Transitioner):
            raise ValueError("A Transitioner is required")
        self.gap_time = gap_time
        self.time_resolution = time_resolution

    def __call__(self, blocks, schedule):
        """
        Schedule a set of `~astroplan.scheduling.ObservingBlock` objects.
        Parameters
        ----------
        blocks : list of `~astroplan.scheduling.ObservingBlock` objects
            The observing blocks to schedule.  Note that the input
            `~astroplan.scheduling.ObservingBlock` objects will *not* be
            modified - new ones will be created and returned.
        schedule : `~astroplan.scheduling.Schedule` object
            A schedule that the blocks will be scheduled in. At this time
            the ``schedule`` must be empty, only defined by a start and
            end time.
        Returns
        -------
        schedule : `~astroplan.scheduling.Schedule`
            A schedule objects which consists of `~astroplan.scheduling.Slot`
            objects with and without populated ``block`` objects containing either
            `~astroplan.scheduling.TransitionBlock` or `~astroplan.scheduling.ObservingBlock`
            objects with populated ``start_time`` and ``end_time`` or ``duration`` attributes
        """
        self.schedule = schedule
        self.schedule.observer = self.observer
        # these are *shallow* copies
        copied_blocks = [copy.copy(block) for block in blocks]
        schedule = self._make_schedule(copied_blocks)
        return schedule

    @abstractmethod
    def _make_schedule(self, blocks):
        """
        Does the actual business of scheduling. The ``blocks`` passed in should
        have their ``start_time` and `end_time`` modified to reflect the
        schedule. Any necessary `~astroplan.scheduling.TransitionBlock` should
        also be added.  Then the full set of blocks should be returned as a list
        of blocks, along with a boolean indicating whether or not they have been
        put in order already.
        Parameters
        ----------
        blocks : list of `~astroplan.scheduling.ObservingBlock` objects
            Can be modified as it is already copied by ``__call__``
        Returns
        -------
        schedule : `~astroplan.scheduling.Schedule`
            A schedule objects which consists of `~astroplan.scheduling.Slot`
            objects with and without populated ``block`` objects containing either
            `~astroplan.scheduling.TransitionBlock` or `~astroplan.scheduling.ObservingBlock`
            objects with populated ``start_time`` and ``end_time`` or ``duration`` attributes.
        """
        raise NotImplementedError
        return schedule

    @classmethod
    @u.quantity_input(duration=u.second)
    def from_timespan(cls, center_time, duration, **kwargs):
        """
        Create a new instance of this class given a center time and duration.
        Parameters
        ----------
        center_time : `~astropy.time.Time`
            Mid-point of time-span to schedule.
        duration : `~astropy.units.Quantity` or `~astropy.time.TimeDelta`
            Duration of time-span to schedule
        """
        start_time = center_time - duration / 2.
        end_time = center_time + duration / 2.
        return cls(start_time, end_time, **kwargs)


In [2480]:
# This is the SequentialScheduler class from astroplan 0.4
# Adding "masks_used" as a class attribute.
class SequentialScheduler(Scheduler):
    """
    A scheduler that does "stupid simple sequential scheduling".  That is, it
    simply looks at all the blocks, picks the best one, schedules it, and then
    moves on.
    """
    # The list "masks_used" can be local to this class,
    # The structure "stats" needs to be global since it is 
    # updated here, but is used by constraint calculations,
    # based upon those updated values. 
    def __init__(self, masks_used=None, *args, **kwargs):
        super(SequentialScheduler, self).__init__(*args, **kwargs)
        # This class attribute keeps track of which masks have been
        # scheduling as the new schedule is computed.
        # Its initial value can either be an empty list or
        # a list with mask_id's [110, 111].  (110 == imaging,
        # 111 = Longslit1)  These two masks are normally on Binospec.
        # There may be a special case where them would be removed.
        # An emply masks_used list would be used in that case.
        self.masks_used = masks_used

    def _make_schedule(self, blocks):
        pre_filled = np.array([[block.start_time, block.end_time] for
                               block in self.schedule.scheduled_blocks])
        if len(pre_filled) == 0:
            a = self.schedule.start_time
            filled_times = Time([a - 1*u.hour, a - 1*u.hour,
                                 a - 1*u.minute, a - 1*u.minute])
            pre_filled = filled_times.reshape((2, 2))
        else:
            filled_times = Time(pre_filled.flatten())
            pre_filled = filled_times.reshape((int(len(filled_times)/2), 2))
        for b in blocks:
            
                
            if b.constraints is None:
                b._all_constraints = self.constraints
            else:
                b._all_constraints = self.constraints + b.constraints
            # to make sure the scheduler has some constraint to work off of
            # and to prevent scheduling of targets below the horizon
            # TODO : change default constraints to [] and switch to append
            if b._all_constraints is None:
                b._all_constraints = [AltitudeConstraint(min=0 * u.deg)]
                b.constraints = [AltitudeConstraint(min=0 * u.deg)]
            elif not any(isinstance(c, AltitudeConstraint) for c in b._all_constraints):
                b._all_constraints.append(AltitudeConstraint(min=0 * u.deg))
                if b.constraints is None:
                    b.constraints = [AltitudeConstraint(min=0 * u.deg)]
                else:
                    b.constraints.append(AltitudeConstraint(min=0 * u.deg))
                    
            if test_SequentialScheduler:
                print("All Constraints: ", repr(b._all_constraints))
                
            b._duration_offsets = u.Quantity([0*u.second, b.duration/2,
                                              b.duration])
            b.observer = self.observer
        current_time = self.schedule.start_time
        
        if test_SequentialScheduler:
            print("Current time: ", repr(current_time))
            # print("Current time: ", repr(current_time), ", blocks:", repr(blocks))
        
        while (len(blocks) > 0) and (current_time < self.schedule.end_time):
            t = current_time.datetime
            txt = str(Time.now()) + ", Begin scheduling for: " + str(t)
            to_stdout(txt)
            
            # new code to skip through the daylight hours at the MMT...
            # if it's after 7AM MST, jump to the next sunset time (+5 minutes)
            if t.hour >= 14:
                current_time = self.observer.sun_set_time(current_time, \
                        which='next', \
                        horizon=configs['max_solar_altitude']*u.deg)
                
                ##### This code allows a gap to be inserted into a queue schedule. 

                #####!!!!!! This is a hard-coded hack.  If it's May 16, advance scheduling to June 2 for the May-June 2017 MMIRS run.
                if t.year == 2017 and t.month == 5 and t.day == 16:
                    # Advancing 16.5 days.
                    current_time += 60*60*24*16.5*u.second
                    # Get the next sunset
                    current_time = self.observer.sun_set_time(current_time, \
                            which='next', \
                            horizon=configs['max_solar_altitude']*u.deg)
                #####!!!!!! End of hard-coded hack.

                # Adding a five--minute "buffer" after sunset.
                # We need to do this so that we don't fail the "max_solar_altitude" criteria
                # at the beginning of the night.
                current_time += 300*u.second
                current_time.format = 'isot'
                txt = str(Time.now()) + ", Advancing scheduling to: " + \
                            str(current_time.datetime)
                to_stdout(txt)
            
            # first compute the value of all the constraints for each block
            # given the current starting time
            if test_SequentialScheduler:
                print("Current time: ", repr(current_time), ", blocks:", repr(blocks))
            
            block_transitions = []
            block_constraint_results = []
            constraint_scores = []
            for b in blocks:
                # first figure out the transition
                if len(self.schedule.observing_blocks) > 0:
                    trans = self.transitioner(
                        self.schedule.observing_blocks[-1], b, current_time, self.observer)
                else:
                    trans = None
                block_transitions.append(trans)
                transition_time = 0*u.second if trans is None else trans.duration

                times = current_time + transition_time + b._duration_offsets

                # make sure it isn't in a pre-filled slot
                if (any((current_time < filled_times) & (filled_times < times[2])) or
                        any(abs(pre_filled.T[0]-current_time) < 1*u.second)):
                    block_constraint_results.append(0)

                else:
                    constraint_res = []
                    for constraint in b._all_constraints:
                        if False:
                            constraint_res.append(constraint(
                                self.observer, b.target, times)) 
                        else:
                            # Code re-organization, a little.                        
                            key = get_key(constraint,b.target,times[0],time2=times[-1])
                            obj = get_score(key)
                            if obj is not None:
                                if False:
                                    print("obj: ", repr(obj))
                                score = float(obj['value'])
                                if verbose2:
                                    print("key:", key, ", using saved score:", score)  
                            else:
                                c = constraint(self.observer, b.target, times)
                                score = np.prod(c)
                                set_score(key,score)
                            if verbose3:
                                print(key, ": ", score)    
                            constraint_res.append(score)
                                
                    # take the product over all the constraints *and* times
                    overall_score = np.prod(constraint_res)
                    block_constraint_results.append(overall_score)
                    block_scores.append({"overall_score":overall_score,"name":b.target.name})
                    if verbose:
                        txt = "***** Target: " + b.target.name.ljust(25) +  " overall_score: " + str(round(overall_score,8)).ljust(12) + " *****"
                        to_stdout(txt)

            # now identify the block that's the best
            bestblock_idx = np.argmax(block_constraint_results)
            txt = "***** Best result: " + str(block_constraint_results[bestblock_idx]) + " *****"
            to_stdout(txt)
            txt = ""
            to_stdout(txt)
                        
            if block_constraint_results[bestblock_idx] == 0.:
                # if even the best is unobservable, we need a gap
                current_time += self.gap_time
                txt = "Adding gap_time", str(self.gap_time)
                to_stdout(txt)
            else:
                # If there's a best one that's observable, first get its transition
                trans = block_transitions.pop(bestblock_idx)
                if trans is not None:
                    self.schedule.insert_slot(trans.start_time, trans)
                    current_time += trans.duration

                # now assign the block itself times and add it to the schedule
                newb = blocks.pop(bestblock_idx)
                newb.start_time = current_time
                current_time += newb.duration
                newb.end_time = current_time
                newb.constraints_value = block_constraint_results[bestblock_idx]

                
                # This is a modification from the previous version of the code.
                
                if current_time > self.schedule.end_time:
                    # We need to append the block back onto the blocks list
                    # since it was popped off and we didn't schedule it.
                    blocks.append(newb)
                    # Do we need to set the current time back as well???
                    # Will this put us into an endless loop of popping and appending the same block???
                    # current_time -= newb.duration
                    # We can't insert this block, so bail out.
                    break               
                
                else:
                    self.schedule.insert_slot(newb.start_time, newb)

                    # This is where new code starts.  We need to publish
                    # results of scheduling while a new schedule is being
                    # computed.

                    # Inserting print information on block being inserted.  JDG 2016-11-30
                    txt = str(Time.now())  +", Scheduling " + newb.target.name + \
                            " from " + str(newb.start_time) + " to " + str(newb.end_time)
                    to_stdout(txt)   

                    # Adding this newblock into time used for the correct program.
                    """
                    "program_hours_allocated": 5.99,
                    "total_hours_requested": 7.5,
                    "total_hours_used": 0,
                    """
                    name = newb.target.name
                    program = get_program(name)
                    dt = (newb.end_time.unix - newb.start_time.unix)/3600.0
                    if test_SequentialScheduler:
                        print("name: ", name)
                        print("program: ", program)
                        print("dt: ", dt)
                        # This is before (i.e., "pre") adding in the time for the block
                        print("total_hours_used (pre): ", stats[program]['total_hours_used'])

                    # Add time to stats for program (n hours)
                    stats[program]['total_hours_used'] += dt
                    if test_SequentialScheduler:
                        # This is after (i.e., "post") adding in the time for the block
                        print("total_hours_used (post): ", stats[program]['total_hours_used'])

                    # Add the mask_id associated this block into "masks_used"
                    # The global masks_used list is used by MaskNumberConstraint
                    mask_id = get_mask_id(name)
                    if mask_id not in self.masks_used:
                        if test_SequentialScheduler:
                                print("Appending to masks_used: ", mask_id, repr(masks_used))
                        self.masks_used.append(mask_id)

            
            sorted_block_scores = sorted(block_scores, key=lambda x: x['overall_score'], reverse=True)
            dispatcher_list = []
            rank = 0
            for bs in sorted_block_scores:
                rank += 1
                name = bs['name']
                field = None
                for f in fields:
                    # HERE2 !!!
                    # if f['block_id'] + '_' + f['objid'] == name:
                    my_name = f['objid'] + '_' +f['block_id'] + "_P" + str(f['pi_priority'])
                    if my_name == name:
                        field = f
                        break
                # Check on this.
                altaz = field.coord.transform_to(AltAz(obstime=Time(t),location=scheduler.earth_location))
                ra_dec = field.coord.to_string("hmsdms").split()
                ra = ra_dec[0]
                dec = ra_dec[1]
             
                # With the short-circuit of calculations, some values may not be defined.
                # Force them to be None.
                my_keys = ["TimeAllocationConstraint",
                           "MeridianConstraint",
                           "PIPriorityConstraint",
                           "AirmassConstraint",
                           "AltitudeConstraint",
                           "AtNightConstraint",
                           'MaskAngleConstraint',
                           'MaskNumberConstraint',
                           "MoonSeparationConstraint",
                           "RotatorConstraint",
                           "TimeConstraint"]

                for k in my_keys:
                    if k not in constraint_scores:
                        scheduler.targets[name].constraint_scores[k] = '' 


                # Build up a nested structure for JSON and Redis.
                data = {"rank":rank, \
                  "timestamp":int(time.time()), \
                  "update":strftime('%Y-%m-%d %H:%M:%S') + " UTC", \
                  "name":name, \
                  "start_time":str(t) + " UTC", \
                  "ra":ra, \
                  "dec":dec, \
                  "pi_priority":field['pi_priority'], \
                  "tac_priority":field['tac_priority'], \
                  "duration":round(field['duration']/60.0,2),
                  # "overhead":field['overhead'],
                  "block_id":field['block_id'],
                  "objid_id":field['objid_id'],
                  "alt":str(altaz.alt), \
                  "az":str(altaz.az), \
                  "TimeAllocationConstraint": scheduler.targets[name].constraint_scores["TimeAllocationConstraint"], \
                  "MeridianConstraint": scheduler.targets[name].constraint_scores["MeridianConstraint"], \
                  "PIPriorityConstraint": scheduler.targets[name].constraint_scores["PIPriorityConstraint"], \
                   # "TACPriorityConstraint": scheduler.targets[name].constraint_scores["TACPriorityConstraint"], \
                  "AirmassConstraint": scheduler.targets[name].constraint_scores["AirmassConstraint"], \
                  "AltitudeConstraint": scheduler.targets[name].constraint_scores["AltitudeConstraint"], \
                  "AtNightConstraint": scheduler.targets[name].constraint_scores["AtNightConstraint"], \
                  "MoonSeparationConstraint": scheduler.targets[name].constraint_scores["MoonSeparationConstraint"], \
                  "RotatorConstraint": scheduler.targets[name].constraint_scores["RotatorConstraint"], \
                  "TimeConstraint": scheduler.targets[name].constraint_scores["TimeConstraint"], \
                  "overall_score":round(bs['overall_score'],8) }
                
                dispatcher_list.append(data)
            
            
            
            # Add this "dispatcher" constraint structure to the growing list.
            constraint_details.append(dispatcher_list)

            # Done if Dispatcher mode
            if scheduler.conf['mode'] == 2:
                to_stdout("Pushing dispatcher results to Redis")
                to_output(dispatcher_list)
                to_status('stopped')
                to_stdout("Dispatcher stopped")
                # Trigger logging the dispatcher output to MySQL
                to_mysql(schedule_id)
                break
                
        # Scheduler mode
        if mode == 1:
            to_stdout("Pushing constraint time series results to Redis")
            to_constraint_details(constraint_details)
        
        return self.schedule


In [2481]:
def main():
    """ 
        Entry point for overall execution.
    """
    global mode
    global instrument
    
    # Setting default values here.  These will typically be
    # command line arguments.
    mode = 2
    instrument = "binospec"
    
    read_setup()
    add_observer()
    add_global_constraints()
    add_block_constraints()    
    create_transitioner()   
    create_scheduler()
    run_scheduler()
    return

In [2482]:
def create_scheduler():
    global seq_scheduler
    seq_scheduler = SequentialScheduler(constraints = global_constraints,
                        observer = mmto,
                        transitioner = transitioner,
                        masks_used=masks_used)
    if True: 
        print("seq_scheduler: ", repr(seq_scheduler))

In [2483]:
def run_scheduler():
    global sequential_schedule
    block_scores = []
    

In [2484]:
    st = datetime.datetime.now()
    print("start time: ", st)

start time:  2018-01-30 12:26:14.820964


In [2485]:
    sequential_schedule = Schedule(start_date, end_date)
    if True: 
        print(get_now(), "sequential_schedule: ", repr(sequential_schedule))

2018-01-30 12:26:14.827689 sequential_schedule:  Schedule containing 0 observing blocks between 2018-02-06 19:00:00.000 and 2018-02-19 19:00:00.000


In [2486]:
    s = seq_scheduler(blocks, sequential_schedule)
    if True: 
        print(get_now(), "seq_scheduler: ", repr(s))

2018-01-30 19:26:14.838161, Begin scheduling for: 2018-02-06 19:00:00
2018-01-30 19:26:14.918578, Advancing scheduling to: 2018-02-07 02:01:49.558801
2018-01-30 12:26:15.005367 : scheduling  <astroplan.scheduling.ObservingBlock (J0803+3138*111*8102*544*P1, 2018-02-07T02:01:49.559 to 2018-02-07T03:31:49.559) at 0x10e084630>
2018-01-30 19:26:15.006019, Begin scheduling for: 2018-02-07 03:31:49.558801
2018-01-30 12:26:15.564895 : scheduling  <astroplan.scheduling.ObservingBlock (J0411-0907*131*8105*544*P2, 2018-02-07T03:32:58.256 to 2018-02-07T06:02:58.256) at 0x10e0848d0>
2018-01-30 19:26:15.565517, Begin scheduling for: 2018-02-07 06:02:58.255879
2018-01-30 12:26:16.087231 : scheduling  <astroplan.scheduling.ObservingBlock (J1135+5011*131*8101*544*P1, 2018-02-07T06:04:48.601 to 2018-02-07T08:34:48.601) at 0x10e084780>
2018-01-30 19:26:16.087848, Begin scheduling for: 2018-02-07 08:34:48.600949
2018-01-30 12:26:16.587173 : scheduling  <astroplan.scheduling.ObservingBlock (Coma3*243*8096*

2018-01-30 19:26:35.419291, Begin scheduling for: 2018-02-09 13:11:32.342493
2018-01-30 19:26:35.548146, Begin scheduling for: 2018-02-09 13:16:32.342493
2018-01-30 19:26:35.677131, Begin scheduling for: 2018-02-09 13:21:32.342493
2018-01-30 19:26:35.801342, Begin scheduling for: 2018-02-09 13:26:32.342493
2018-01-30 19:26:35.927048, Begin scheduling for: 2018-02-09 13:31:32.342493
2018-01-30 19:26:36.053365, Begin scheduling for: 2018-02-09 13:36:32.342493
2018-01-30 19:26:36.184155, Begin scheduling for: 2018-02-09 13:41:32.342493
2018-01-30 19:26:36.306048, Begin scheduling for: 2018-02-09 13:46:32.342493
2018-01-30 19:26:36.433996, Begin scheduling for: 2018-02-09 13:51:32.342493
2018-01-30 19:26:36.562757, Begin scheduling for: 2018-02-09 13:56:32.342493
2018-01-30 19:26:36.693382, Begin scheduling for: 2018-02-09 14:01:32.342493
2018-01-30 19:26:36.772915, Advancing scheduling to: 2018-02-10 02:04:12.709544
2018-01-30 19:26:36.894246, Begin scheduling for: 2018-02-10 02:09:12.709

In [2487]:
    et = datetime.datetime.now()
    print("end time: ", et)

end time:  2018-01-30 12:26:39.800901


In [2488]:
    time_diff = et - st
    print("Elapsed time: ", str(time_diff))

Elapsed time:  0:00:24.979937


In [2489]:
"""  
This is the schedule for the two following time evaluations:
2018-01-27 10:07:34.649577 : scheduling  <astroplan.scheduling.ObservingBlock (stream_tri_03_7982_P1, 2017-12-15 00:51:00.000 to 2017-12-15 02:36:00.000) at 0x111ccdcc0>
2018-01-27 10:07:38.869270 : scheduling  <astroplan.scheduling.ObservingBlock (jj0151-2_7905_P1, 2017-12-15 02:36:19.166 to 2017-12-15 03:26:19.166) at 0x115574b38>
2018-01-27 10:07:43.259979 : scheduling  <astroplan.scheduling.ObservingBlock (PS16fgt (copy)_7753_P1, 2017-12-15 03:26:31.772 to 2017-12-15 03:46:31.772) at 0x114fe9eb8>
2018-01-27 10:07:47.742714 : scheduling  <astroplan.scheduling.ObservingBlock (jj0151-1_7904_P1, 2017-12-15 03:46:44.378 to 2017-12-15 06:16:44.378) at 0x115574940>
2018-01-27 10:07:51.891498 : scheduling  <astroplana.scheduling.ObservingBlock (SAO-9_L1527IRS (copy)_7874_P1, 2017-12-15 06:17:24.019 to 2017-12-15 07:07:24.019) at 0x111cb5198>
seq_scheduler:  Schedule containing 5 observing blocks between 2017-12-14 22:46:00.000 and 2017-12-15 07:07:24.019
"""

"""
2018-01-27a
Constraint calculations with no stored values in scores.sqlite took:
start: '2018-01-27 08:58:34.161705'
end: 2018-01-27 09:35:11.433527'
Around 36 minutes for 12720 entries in the scores table.
"""

"""
2018-01-27b (rerunning same calculations with saved values)
Constraint calculations with all stored values in scores.sqlite took:
start: '2018-01-27 10:07:26.775868'
end: '2018-01-27 10:07:51.919122'
Around 25 seconds for 12720 entries in the scores table.
"""



"\n2018-01-27b (rerunning same calculations with saved values)\nConstraint calculations with all stored values in scores.sqlite took:\nstart: '2018-01-27 10:07:26.775868'\nend: '2018-01-27 10:07:51.919122'\nAround 25 seconds for 12720 entries in the scores table.\n"

In [2490]:
if __name__ == "__main__":
    redis_client = redis.StrictRedis( host=redis_host, decode_responses=True ) # without that, strings have a 'b' in front
    
    parser = argparse.ArgumentParser(description='mmtscheduler.py')
    parser.add_argument('-m','--mode', help='Scheduling mode.  mode 1: "scheduler" mode; mode 2: "dispatcher" mode.', type=int, choices=[1,2], required=False)
    parser.add_argument('-d','--debug', help='Print debugging messages', required=False)
    parser.add_argument('-v','--verbose', help='Print verbose messages: 0 (no messages), 1 (verbose), 2 (very verbose)', type=int, choices=[0,1,2], required=False)
    parser.add_argument('-s','--start_date', help='Start time for scheduling (UTC time), e.g., "2017-10-01T03:30:00".  Defines the time used for dispatcher mode and the start time for scheduler mode', required=False)
    parser.add_argument('-e','--end_date', help='End time for scheduling (UTC time), e.g., e.g., "2017-10-01T12:30:00".  Defines the end time for scheduler mode.', required=False)
    parser.add_argument('-i','--schedule_id', help='Queue scheduling run id.', type=int, required=True)
    args = vars(parser.parse_args())
    if True:
        print("args: ", repr(args))

usage: ipykernel_launcher.py [-h] [-m {1,2}] [-d DEBUG] [-v {0,1,2}]
                             [-s START_DATE] [-e END_DATE] -i SCHEDULE_ID
ipykernel_launcher.py: error: the following arguments are required: -i/--schedule_id


SystemExit: 2

/Users/jdgibson/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
    print("Here we go!!!")
    main(args)